In [1]:
!pip install pyngrok
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [2]:
from pyngrok import ngrok
import os

# SUBSTITUA "SEU_AUTHTOKEN_REAL_AQUI" PELO SEU TOKEN COPIADO DO NGROK.COM
NGROK_AUTH_TOKEN = "2yugKJtv0k0p9OLDZpJU7nuDKQT_2y2KcX4ARYLAqSqoXuRNP" # <<<<<< AQUI É ONDE VOCÊ COLA O TOKEN
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("ngrok configurado!")

ngrok configurado!


In [3]:
%%writefile /content/drive/MyDrive/Economia/app.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import geopandas as gpd
import os

# Configurações Iniciais do Streamlit
st.set_page_config(layout="wide")
st.title('Poder de Compra do Salário-Mínimo: Uma Comparação Internacional')
st.write("""
Este projeto permite explorar o poder de compra do salário-mínimo em diferentes países,
com foco na quantidade de **alimentos básicos** que podem ser adquiridos.
""")

# Caminhos para os Dados e Mapa
DADOS_DIR = '/content/drive/MyDrive/Economia/dados'
MAPA_DIR = '/content/drive/MyDrive/Economia/mapa'

# Carregamento e Pré-processamento dos Dados
@st.cache_data # Cache para otimizar o carregamento e processamento dos dados
def load_and_process_data():
    # Carregar e pré-processar o arquivo de Alimentos
    try:
        df_alimentos = pd.read_csv(
            os.path.join(DADOS_DIR, "dados_economia(Alimentos).csv"),
            sep=";",
            skiprows=1,
            encoding="ISO-8859-1"
        )
        df_alimentos.dropna(axis=1, how='all', inplace=True)
        df_alimentos.rename(columns={df_alimentos.columns[0]: "Produto"}, inplace=True)
        df_alimentos = df_alimentos.head(10).copy()

        cols_to_convert_alimentos = df_alimentos.columns.drop('Produto')
        for col in cols_to_convert_alimentos:
            df_alimentos[col] = df_alimentos[col].astype(str).str.replace(',', '.', regex=False)
            df_alimentos[col] = pd.to_numeric(df_alimentos[col])

        df_alimentos_long = df_alimentos.melt(id_vars=['Produto'],
                                              var_name='País',
                                              value_name='Preço_USD')

    except Exception as e:
        st.error(f"Erro ao carregar/pré-processar dados_economia(Alimentos).csv: {e}")
        st.stop() # Parar o aplicativo se houver um erro crítico de dados

    # Carregar e pré-processar o arquivo de Salário Mínimo
    try:
        df_salario_minimo = pd.read_csv(
            os.path.join(DADOS_DIR, "dados_economia(salário-mínimo).csv"),
            sep=";",
            encoding="ISO-8859-1"
        )
        df_salario_minimo['salário-mínimo (USD$)'] = df_salario_minimo['salário-mínimo (USD$)'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
        df_salario_minimo['salário-mínimo (USD$)'] = pd.to_numeric(df_salario_minimo['salário-mínimo (USD$)'])
        df_salario_minimo.rename(columns={'Países': 'País'}, inplace=True)

    except Exception as e:
        st.error(f"Erro ao carregar dados_economia(salário-mínimo).csv: {e}")
        st.stop()

    # Carregar e pré-processar o arquivo de PIB
    try:
        df_pib = pd.read_csv(
            os.path.join(DADOS_DIR, "dados_economia(PIB).csv"),
            sep=";",
            encoding="ISO-8859-1"
        )
        cols_to_drop_pib = [col for col in df_pib.columns if 'Unnamed:' in col]
        df_pib.drop(columns=cols_to_drop_pib, inplace=True)
        df_pib.rename(columns={'Países': 'País'}, inplace=True)

    except Exception as e:
        st.error(f"Erro ao carregar dados_economia(PIB).csv: {e}")
        st.stop()

    # Junção dos DataFrames
    df_merged_salario_pib = pd.merge(df_salario_minimo, df_pib, on='País', how='inner')
    df_final = pd.merge(df_alimentos_long, df_merged_salario_pib, on='País', how='inner')

    # Cálculo do Poder de Compra
    df_final['Poder_Compra_Unidades'] = df_final['salário-mínimo (USD$)'] / df_final['Preço_USD']

    # Análise do Poder de Compra Médio por País
    df_poder_compra_medio_pais = df_final.groupby('País')['Poder_Compra_Unidades'].mean().reset_index()
    df_poder_compra_medio_pais = df_poder_compra_medio_pais.sort_values(by='Poder_Compra_Unidades', ascending=False)

    # Definindo categorias de países
    paises_desenvolvidos = ['Alemanha', 'Estados Unidos', 'Japão']
    paises_em_ascensao = ['China', 'Índia']
    paises_emergentes = ['África do Sul', 'Argentina', 'Brasil', 'Indonésia', 'México']

    def categorize_country_final(country):
        if country in paises_desenvolvidos:
            return 'Economias Desenvolvidas'
        elif country in paises_em_ascensao:
            return 'Grandes Economias em Ascensão'
        elif country in paises_emergentes:
            return 'Economias de Mercado Emergentes'
        else:
            return 'Outro'

    df_final['Tipo_País'] = df_final['País'].apply(categorize_country_final)
    df_poder_compra_medio_pais['Tipo_País'] = df_poder_compra_medio_pais['País'].apply(categorize_country_final)

    return df_final, df_salario_minimo, df_poder_compra_medio_pais

# Carregar os dados uma única vez usando a função cacheada
df_final, df_salario_minimo, df_poder_compra_medio_pais = load_and_process_data()

# Barra Lateral para Filtros
st.sidebar.header('Filtros')

# Filtro por Categoria
categorias_selecionadas = st.sidebar.multiselect(
    'Selecione as Categorias de Países:',
    df_final['Tipo_País'].unique(),
    default=df_final['Tipo_País'].unique()
)

# Filtro por País
paises_disponiveis = df_final[df_final['Tipo_País'].isin(categorias_selecionadas)]['País'].unique()
paises_selecionados = st.sidebar.multiselect(
    'Selecione os Países:',
    paises_disponiveis,
    default=paises_disponiveis
)

# Aplica ambos os filtros
df_filtrado = df_final[df_final['Tipo_País'].isin(categorias_selecionadas) & df_final['País'].isin(paises_selecionados)]
df_poder_compra_medio_pais_filtrado = df_poder_compra_medio_pais[df_poder_compra_medio_pais['Tipo_País'].isin(categorias_selecionadas) & df_poder_compra_medio_pais['País'].isin(paises_selecionados)]

# Visualização 1: Salário Mínimo Mensal por País (Gráfico de Barras)
st.header('Salário Mínimo Mensal por País (em USD$)')
st.subheader('Comparação Bruta dos Salários Mínimos')

df_salario_minimo_filtered = df_salario_minimo[df_salario_minimo['País'].isin(paises_selecionados)]
if not df_salario_minimo_filtered.empty:
    fig_salario, ax_salario = plt.subplots(figsize=(12, 7))
    df_salario_minimo_sorted = df_salario_minimo_filtered.sort_values(by='salário-mínimo (USD$)', ascending=False)
    sns.barplot(x='salário-mínimo (USD$)', y='País', data=df_salario_minimo_sorted, palette='cubehelix', ax=ax_salario)
    ax_salario.set_title('Salário Mínimo Mensal por País (em USD$)', fontsize=16)
    ax_salario.set_xlabel('Salário Mínimo (USD$)', fontsize=12)
    ax_salario.set_ylabel('País', fontsize=12)
    ax_salario.grid(axis='x', linestyle='--')
    plt.tight_layout()
    st.pyplot(fig_salario)
    plt.clf() # Limpar figura para evitar sobreposição
    plt.close(fig_salario)
else:
    st.warning("Nenhum país selecionado para exibir o gráfico de Salário Mínimo.")

# Visualização 2: Poder de Compra Médio por País (Ranking Geral)
st.header('Poder de Compra Médio de Alimentos por Salário Mínimo')
st.subheader('Classificação por Quantidade Média de Produtos Adquiríveis')

if not df_poder_compra_medio_pais_filtrado.empty:
    fig_compra_medio, ax_compra_medio = plt.subplots(figsize=(12, 7))
    sns.barplot(x='Poder_Compra_Unidades', y='País', data=df_poder_compra_medio_pais_filtrado.sort_values('Poder_Compra_Unidades', ascending=False), palette='viridis', ax=ax_compra_medio)
    ax_compra_medio.set_title('Poder de Compra Médio de Alimentos por Salário Mínimo (Ranking Geral)', fontsize=16)
    ax_compra_medio.set_xlabel('Poder de Compra Médio (Unidades de Produto)', fontsize=12)
    ax_compra_medio.set_ylabel('País', fontsize=12)
    ax_compra_medio.grid(axis='x', linestyle='--')
    plt.tight_layout()
    st.pyplot(fig_compra_medio)
    plt.clf()
    plt.close(fig_compra_medio)
else:
    st.warning("Nenhum país selecionado para exibir o gráfico de Poder de Compra Médio.")

# Visualização 3: Poder de Compra de produtos selecionados por País
st.header('Poder de Compra Detalhado por Produto e País')
st.subheader('Quantas Unidades de Cada Produto o Salário Mínimo Compra')

if not df_filtrado.empty:
    fig_prod, ax_prod = plt.subplots(figsize=(16, 10))
    sns.barplot(x='Poder_Compra_Unidades', y='País', hue='Produto', data=df_filtrado.sort_values(by=['Produto', 'Poder_Compra_Unidades'], ascending=[True, False]), palette='tab20', ax=ax_prod)
    ax_prod.set_title('Poder de Compra de Produtos por Salário Mínimo (em Unidades de Produto)', fontsize=16)
    ax_prod.set_xlabel('Poder de Compra (Unidades de Produto)', fontsize=12)
    ax_prod.set_ylabel('País', fontsize=12)
    ax_prod.grid(axis='x', linestyle='--')
    ax_prod.legend(title='Produto', bbox_to_anchor=(1.05, 1), loc='upper left', ncol=1)
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    st.pyplot(fig_prod)
    plt.clf()
    plt.close(fig_prod)
else:
    st.warning("Nenhum país ou produto selecionado para exibir o gráfico de Poder de Compra por Produto.")

# Visualização 4: Relação entre Poder de Compra Médio e PIB Per Capita
st.header('Poder de Compra Médio de Alimentos vs. PIB Per Capita')
st.subheader('Investigando a Correlação entre Renda e Capacidade de Compra')

try:
    df_pib = pd.read_csv(
        os.path.join(DADOS_DIR, "dados_economia(PIB).csv"),
        sep=";",
        encoding="ISO-8859-1"
    )
    cols_to_drop_pib = [col for col in df_pib.columns if 'Unnamed:' in col]
    df_pib.drop(columns=cols_to_drop_pib, inplace=True)
    df_pib.rename(columns={'Países': 'País'}, inplace=True)
except Exception as e:
    st.error(f"Erro ao carregar dados_economia(PIB).csv para o gráfico de correlação: {e}")
    df_pib = pd.DataFrame({'País': [], 'Per capita(USD$)': []}) # DataFrame vazio para evitar erros

df_pib_poder_compra = pd.merge(df_poder_compra_medio_pais_filtrado, df_pib[['País', 'Per capita(USD$)']], on='País', how='inner')

if not df_pib_poder_compra.empty:
    fig_scatter, ax_scatter = plt.subplots(figsize=(10, 6))
    sns.scatterplot(x='Per capita(USD$)', y='Poder_Compra_Unidades', hue='País', data=df_pib_poder_compra, s=100, ax=ax_scatter)
    ax_scatter.set_title('Poder de Compra Médio de Alimentos vs. PIB Per Capita', fontsize=16)
    ax_scatter.set_xlabel('PIB Per Capita (USD$)', fontsize=12)
    ax_scatter.set_ylabel('Poder de Compra Médio (Unidades de Produto)', fontsize=12)
    ax_scatter.grid(True, linestyle='--')

    # Adiciona rótulos de texto aos pontos
    for i, row in df_pib_poder_compra.iterrows():
        ax_scatter.text(row['Per capita(USD$)'] + (ax_scatter.get_xlim()[1] * 0.01), row['Poder_Compra_Unidades'], row['País'], fontsize=9)

    plt.tight_layout()
    st.pyplot(fig_scatter)
    plt.clf()
    plt.close(fig_scatter)
else:
    st.warning("Nenhum país selecionado ou dados de PIB insuficientes para exibir o gráfico de Poder de Compra vs. PIB.")

# Visualização 5: Poder de Compra Médio por Categoria de País
st.header('Grouping: Poder de Compra Médio por Categoria de País')
st.subheader('Análise Comparativa entre os Diferentes Grupos Econômicos')

df_poder_compra_por_tipo_final = df_filtrado.groupby('Tipo_País')['Poder_Compra_Unidades'].mean().reset_index()
ordem_categorias_final = ['Economias Desenvolvidas', 'Grandes Economias em Ascensão', 'Economias de Mercado Emergentes']
df_poder_compra_por_tipo_final['Tipo_País'] = pd.Categorical(df_poder_compra_por_tipo_final['Tipo_País'], categories=ordem_categorias_final, ordered=True)
df_poder_compra_por_tipo_final = df_poder_compra_por_tipo_final.sort_values('Tipo_País')

if not df_poder_compra_por_tipo_final.empty:
    fig_cat, ax_cat = plt.subplots(figsize=(10, 6))
    sns.barplot(x='Tipo_País', y='Poder_Compra_Unidades', data=df_poder_compra_por_tipo_final, palette='coolwarm', ax=ax_cat)
    ax_cat.set_title('Poder de Compra Médio de Alimentos por Categoria de País', fontsize=16)
    ax_cat.set_xlabel('Categoria de País', fontsize=12)
    ax_cat.set_ylabel('Poder de Compra Médio (Unidades de Produto)', fontsize=12)
    ax_cat.grid(axis='y', linestyle='--')
    plt.xticks(rotation=15, ha='right') # Ajuste para melhor visualização dos rótulos
    plt.tight_layout()
    st.pyplot(fig_cat)
    plt.clf()
    plt.close(fig_cat)
else:
    st.warning("Nenhuma categoria selecionada ou dados insuficientes para exibir o gráfico de Poder de Compra por Categoria.")

# Visualização 6: Posição do Brasil vs. Novas Categorias (Poder de Compra GERAL)
st.header('Brasil no Cenário Global: Poder de Compra por Categoria de País')
st.subheader('Como o Brasil se Compara aos Diferentes Grupos Econômicos')

if not df_poder_compra_medio_pais_filtrado.empty:
    fig_brasil, ax_brasil = plt.subplots(figsize=(10, 7))
    sns.barplot(x='Poder_Compra_Unidades', y='País', hue='Tipo_País', data=df_poder_compra_medio_pais_filtrado.sort_values(by='Poder_Compra_Unidades', ascending=False), palette='viridis', dodge=False, ax=ax_brasil)
    ax_brasil.set_title('Poder de Compra Médio Geral por Salário Mínimo por Categoria de País', fontsize=16)
    ax_brasil.set_xlabel('Poder de Compra Médio (Unidades de Produto)', fontsize=12)
    ax_brasil.set_ylabel('País', fontsize=12)
    ax_brasil.grid(axis='x', linestyle='--')
    ax_brasil.legend(title='Categoria de País', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    st.pyplot(fig_brasil)
    plt.clf()
    plt.close(fig_brasil)
else:
    st.warning("Nenhum país selecionado para exibir o gráfico de Posição do Brasil.")

# Visualização 7: Mapa Mundial
st.header('Mapa Mundial do Poder de Compra Médio')
st.subheader('Distribuição Geográfica do Poder de Compra do Salário Mínimo')

# Carregar dados do mapa mundial
try:
    world_map = gpd.read_file(os.path.join(MAPA_DIR, "ne_110m_admin_0_countries.shp"))

except Exception as e:
    st.error(f"Erro ao carregar o mapa mundial de 'ne_110m_admin_0_countries.shp': {e}")
    st.info(f"Por favor, verifique se 'ne_110m_admin_0_countries.shp' (e os demais arquivos do shapefile: .shx, .dbf, .prj, .cpg) estão na pasta correta: `{MAPA_DIR}` e o nome está exato.")
    st.stop() # Parar o aplicativo se o mapa não puder ser carregado

# Padronizar nomes de países para a junção
nome_paises_mapa = {
    'África do Sul': 'South Africa',
    'Alemanha': 'Germany',
    'Argentina': 'Argentina',
    'Brasil': 'Brazil',
    'China': 'China',
    'Estados Unidos': 'United States of America',
    'Índia': 'India',
    'Indonésia': 'Indonesia',
    'Japão': 'Japan',
    'México': 'Mexico'
}

# Criar uma nova coluna no df_poder_compra_medio_pais com os nomes do mapa
df_poder_compra_medio_pais_para_mapa = df_poder_compra_medio_pais.copy()
df_poder_compra_medio_pais_para_mapa['País_Mapa'] = df_poder_compra_medio_pais_para_mapa['País'].map(nome_paises_mapa)

# Juntar os dados do poder de compra médio com os dados do mapa
mapa_poder_compra = world_map.set_index('NAME_EN').merge(
    df_poder_compra_medio_pais_para_mapa.set_index('País_Mapa'),
    left_index=True,
    right_index=True,
    how='left'
)

# Plotar o mapa
fig_map, ax_map = plt.subplots(1, 1, figsize=(18, 12))

ax_map.set_xticks([])
ax_map.set_yticks([])

mapa_poder_compra.plot(column='Poder_Compra_Unidades',
                        cmap='Reds',
                        linewidth=0.5,
                        ax=ax_map,
                        edgecolor='gray',
                        legend=True,
                        missing_kwds={"color": "lightgrey",
                                      "label": "Sem Dados"},
                        legend_kwds={'label': "Poder de Compra Médio (Unidades de Produto)",
                                     'orientation': "horizontal"})

ax_map.set_title('Mapa Mundial do Poder de Compra Médio do Salário Mínimo (em Unidades de Produto)', fontsize=16)
ax_map.axis('off')
plt.tight_layout()
st.pyplot(fig_map) # Exibir o mapa no Streamlit
plt.clf()
plt.close(fig_map)

# Seção de Perguntas de Pesquisa e Análise
st.header('Perguntas de Pesquisa e Análise')

st.markdown("### 1. Qual a quantidade de alimentos básicos que o salário-mínimo compra em diferentes países?")
st.write("""
A variação na quantidade de cestas básicas que podem ser adquiridas é drástica. Enquanto a Alemanha pode comprar mais de 1200 unidades médias de produtos, a Argentina mal alcança 250 unidades. Isso reflete as enormes disparidades econômicas e de custo de vida entre os países. A Indonésia se destaca pelo grande poder de compra de produtos básicos como arroz e banana, o que pode ser reflexo de uma produção local abundante desses itens e de um custo de vida relativamente baixo para esses produtos específicos em relação ao salário mínimo.
""")

st.markdown("### 2. Existe uma correlação clara entre o PIB per capita de um país e o poder de compra de alimentos do salário-mínimo?")
st.write("""
Existe uma tendência geral positiva: países com PIB per capita mais alto tendem a ter maior poder de compra de alimentos com o salário-mínimo.

- Alemanha lidera em ambos os indicadores.
- EUA, embora ricos, têm poder de compra bem inferior, indicando influência de outros fatores, como custo de vida e política salarial.
- Índia e Argentina, com PIBs baixos, têm também baixo poder de compra.

**Exceções importantes:**
- Indonésia se destaca com PIB baixo, mas alto poder de compra, sugerindo alimentos baratos ou salário-mínimo mais eficaz.
- Japão, com PIB alto, mas poder de compra modesto, possivelmente por custo elevado dos alimentos.
- Brasil, China e África do Sul estão em posição intermediária, mostrando certa consistência entre renda e poder de compra entre emergentes.
""")

st.markdown("### 3. Como o Brasil se compara a economias desenvolvidas e a outras economias emergentes?")
st.write("""
O Brasil tem um desempenho inferior às economias desenvolvidas (como Alemanha e EUA), tanto em PIB per capita quanto em poder de compra de alimentos. Embora o Japão tenha alto PIB, seu poder de compra é mais próximo ao de países como a Indonésia.

Entre os emergentes, o Brasil ocupa uma posição intermediária. Está acima de Índia, México e Argentina, mas abaixo de Indonésia e China, que têm desempenho superior mesmo com PIBs semelhantes.

O Brasil se encaixa na média ou levemente abaixo da média entre os emergentes, com cerca de 460 unidades médias de alimentos compradas com o salário-mínimo.

**Fatores que influenciam essa posição incluem:**
- Inflação, que reduz o poder de compra;
- Produtividade, que impacta os salários reais;
- Políticas salariais, que definem o valor do mínimo;
- Estrutura econômica, que afeta preços e salários;
- Custo de vida, que pode neutralizar salários mais altos.
""")

st.markdown("### 4. Qual a diferença no poder de compra médio de alimentos entre as categorias de países ('Desenvolvidos', 'Em Ascensão' e 'Emergentes')?")
st.write("""
Desenvolvidos: Apresentam o maior poder de compra médio, com cerca de 850 unidades de alimentos. A Alemanha se destaca, mas há variações internas — os EUA, por exemplo, têm desempenho inferior à Alemanha.

Emergentes: Têm poder de compra médio de aproximadamente 460 unidades, com países como Brasil, México e Argentina em posições mais baixas, e Indonésia como exceção positiva.

Em Ascensão: Média próxima aos emergentes (cerca de 440 unidades), com destaque para a China, cujo desempenho se mistura ao dos emergentes.
""")

Overwriting /content/drive/MyDrive/Economia/app.py


In [4]:
from pyngrok import ngrok
import os
import subprocess
import time

# O caminho do seu app.py, que está diretamente na sua pasta Economia
streamlit_app_path = '/content/drive/MyDrive/Economia/app.py'

# Iniciar o servidor Streamlit em segundo plano
# Os parâmetros '--server.enableCORS false' e '--server.enableXsrfProtection false'
# são importantes para garantir o funcionamento correto no ambiente Colab/ngrok.
proc = subprocess.Popen([
    'streamlit', 'run', streamlit_app_path,
    '--server.port', '8501',
    '--server.headless', 'true',
    '--server.enableCORS', 'false',
    '--server.enableXsrfProtection', 'false'
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Dar um tempo para o servidor iniciar. 15 segundos geralmente é suficiente.
time.sleep(15)

# Criar um túnel ngrok para a porta 8501 (porta padrão do Streamlit)
public_url = ngrok.connect("8501", proto="http") # Linha corrigida
print(f"🎉 Seu aplicativo Streamlit está acessível em: {public_url}")
print("\nPara parar o aplicativo e o túnel, pare a execução desta célula (clicando no quadrado ao lado dela no Colab).")

🎉 Seu aplicativo Streamlit está acessível em: NgrokTunnel: "https://10d5-34-9-203-167.ngrok-free.app" -> "http://localhost:8501"

Para parar o aplicativo e o túnel, pare a execução desta célula (clicando no quadrado ao lado dela no Colab).
